In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
import keras 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, BatchNormalization, MaxPooling2D, Dropout
from tensorflow.keras.optimizers.legacy import Adam

In [2]:
train_datagen = ImageDataGenerator(rescale = 1./224,
                                   validation_split = 0.2)

In [3]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [4]:
import pandas as pd

df=pd.read_csv('/Users/kimyoojin/Desktop/졸업과제용/데이터/severstal-steel-defect-detection/train.csv')

In [5]:
df.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


In [6]:
df.tail()

,ImageId,ClassId,EncodedPixels
7090,ffcf72ecf.jpg,3,121911 34 122167 101 122422 169 122678 203 122...
7091,fff02e9c5.jpg,3,207523 3 207777 9 208030 15 208283 22 208537 2...
7092,fffe98443.jpg,3,105929 5 106177 14 106424 24 106672 33 106923 ...
7093,ffff4eaa8.jpg,3,16899 7 17155 20 17411 34 17667 47 17923 60 18...
7094,ffffd67df.jpg,3,30931 43 31103 127 31275 211 31489 253 31745 2...


In [7]:
df.loc[lambda df: df['ImageId']=='0000f269f.jpg']

,ImageId,ClassId,EncodedPixels


In [24]:
sub=pd.read_csv('/Users/kimyoojin/Desktop/졸업과제용/데이터/severstal-steel-defect-detection/sample_submission.csv')

In [9]:
sub.head()

,ImageId,EncodedPixels,ClassId
0,0000f269f.jpg,1 409600,0
1,000ccc2ac.jpg,1 409600,0
2,002451917.jpg,1 409600,0
3,003c5da97.jpg,1 409600,0
4,0042e163f.jpg,1 409600,0


In [10]:
df['ClassId']=df['ClassId'].astype(str)

In [26]:
df.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


In [32]:
df['ClassId']=df['ClassId'].astype(int)
df.loc[1 <= df['ClassId'], 'ClassId'] = '1'

In [35]:
df.loc[0, 'ClassId'] = '0'
df.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,0,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,1,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,1,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,1,229501 11 229741 33 229981 55 230221 77 230468...


In [45]:
trainData = train_datagen.flow_from_dataframe(dataframe=df,
                                                directory='/Users/kimyoojin/Desktop/졸업과제용/데이터/severstal-steel-defect-detection/train_images',
                                                target_size=(1600, 256), x_col='ImageId',
                                                y_col='ClassId', batch_size=16,
                                                class_mode='binary', subset = "training")

Found 5676 validated image filenames belonging to 2 classes.


In [46]:
validData = train_datagen.flow_from_dataframe(dataframe=df,
                                                directory='/Users/kimyoojin/Desktop/졸업과제용/데이터/severstal-steel-defect-detection/train_images',
                                                target_size=(1600, 256), x_col='ImageId',
                                                y_col='ClassId', batch_size=16,
                                                class_mode='binary', subset = "validation")

Found 1419 validated image filenames belonging to 2 classes.


In [47]:
# Xception

base_model = tf.keras.applications.Xception(input_shape=(1600,256,3),include_top=False,weights="imagenet")

# Freezing Layers

for layer in base_model.layers[:-5]:
    layer.trainable=False

# Building Model

model=Sequential()
model.add(base_model)
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(256,activation="relu"))
model.add(Dense(1,activation="softmax"))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 50, 8, 2048)       20861480  
                                                                 
 dropout_15 (Dropout)        (None, 50, 8, 2048)       0         
                                                                 
 flatten_5 (Flatten)         (None, 819200)            0         
                                                                 
 dense_20 (Dense)            (None, 256)               209715456 
                                                                 
 dropout_16 (Dropout)        (None, 256)               0         
                                                                 
 dense_21 (Dense)            (None, 128)               32896     
                                                                 
 dropout_17 (Dropout)        (None, 128)              

In [48]:
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["acc"])  

In [49]:
model.fit(trainData, steps_per_epoch= len(trainData), epochs = 10, verbose= 1, validation_data = validData, validation_steps = len(validData))

Epoch 1/10
127/355 [=========>....................] - ETA: 31:44 - loss: 0.0000e+00 - acc: 1.0000

KeyboardInterrupt: 

In [18]:
test_datagen = ImageDataGenerator(rescale = 1./224)

In [19]:
# test_set = test_datagen.flow_from_directory('/kaggle/input/severstal-steel-defect-detection/test_images/',
#                                             target_size = (128, 128),
#                                             batch_size=16,
#                                            class_mode='categorical',)

In [20]:
import os

In [21]:
image_list = os.listdir('/Users/kimyoojin/Desktop/졸업과제용/데이터/severstal-steel-defect-detection/test_images')

In [22]:
import numpy as np

In [23]:
predicted_classes = []

for image_name in image_list:
    image_path = os.path.join('/Users/kimyoojin/Desktop/졸업과제용/데이터/severstal-steel-defect-detection/test_images', image_name)
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(1600, 256))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = tf.keras.applications.resnet50.preprocess_input(image)
    predictions = model.predict(image)
    predicted_class = np.argmax(predictions, axis=1)
    predicted_classes.append(predicted_class)


1/1 [==============================] - 0s 15ms/step


In [24]:
submission=pd.DataFrame()

In [25]:
values=np.array(predicted_classes)

In [26]:
submission['ImageId']=image_list
submission['EncodedPixels']='1 409600'
submission['ClassId']=values

In [27]:
submission.head()

,ImageId,EncodedPixels,ClassId
0,2b6c68337.jpg,1 409600,2
1,73ff8efef.jpg,1 409600,2
2,1e45cda5b.jpg,1 409600,2
3,549fb29f7.jpg,1 409600,2
4,5f0696eb5.jpg,1 409600,2


In [28]:
submission.to_csv('submission.csv',index=False)

In [ ]:
# submission에 ClassId = 1인 경우에만 

In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [2]:
base_model = ResNet50(include_top=True, input_shape = (224, 224 ,3), weights = 'imagenet')

102967424/102967424 [==============================] - 2s 0us/step


In [ ]:
base_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics = 'acc')

for i in range(5):
  inputs = test_images[i]                 # Test_images의 i번째 이미지에 대한 평가 
  inputs = tf.expand_dims(inputs, axis=0) # ResNet이 원하는 이미지 차원은 (None, 224, 224, 3)이므로 차원 하나를 더 추가해준다.
  x = tf.keras.layers.experimental.preprocessing.Resizing(224, 224)(inputs) # ResNet의 인풋사이즈에 맞춰준다. > 안맞춰도 돌아가지만 성능을 위해
  x = tf.keras.applications.resnet50.preprocess_input(x)  # 이전단계에서 안맞춰도 돌아가는 이유 > 바로 이 전처리 모듈이 있어서 알아서 맞춰주긴함
  base_model.evaluate(x, test_labels[i])  # 모델의 성능을 파악해

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import decode_predictions

for i in range(5):

  inputs = test_images[i]
  inputs = tf.expand_dims(inputs, axis=0)
  x = tf.keras.layers.experimental.preprocessing.Resizing(224, 224)(inputs)
  x = tf.keras.applications.resnet50.preprocess_input(x) 

  preds = base_model.predict(x)  # ResNet Model의 예측진행

  # 예측 실행
  fig= plt.figure(figsize=(3,3))
  plt.title(decode_predictions(preds, top=1)[0][0][1])# 타이틀에 예측 명 1순위를 넣는다.
  plt.imshow(test_images[i])	# 이미지는 
  plt.show();